In [1]:
import sys
import os
import pandas as pd
from datetime import datetime, timedelta
import time

root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from functions import fetch_github_data


from config import GITHUB_TOKEN


# @TODO 
# make the num_stars num_forks and num_releases dynamic
# regen the data
# fine-tuning or transfer learning

/opt/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Github's Personal Access Token
TOKEN = GITHUB_TOKEN

In [3]:
total_iterations = 50
pause_time = 62  
fetch_time = 20
total_estimated_time = total_iterations * (fetch_time + pause_time)
start_time = datetime.now()

headers = {'Authorization': f'token {TOKEN}'}
query = 'AI OR "artificial intelligence"'

data = []

for i in range(1, total_iterations+1):
    print(f"Fetching page {i}")
    data.extend(fetch_github_data(headers, query, pages=i, per_page=20))
    
    elapsed_time = (datetime.now() - start_time).total_seconds()
    remaining_time = total_estimated_time - elapsed_time

    # Display remaining time in HH:MM:SS format
    remaining_time_formatted = str(timedelta(seconds=max(remaining_time, 0)))  # max with 0 to avoid negative times
    print(f"Elapsed time: {str(timedelta(seconds=int(elapsed_time)))}")
    print(f"Estimated remaining time: {remaining_time_formatted}")
    
    # Pause for the specified time
    if i < total_iterations:  # No need to pause after the last iteration
        print(f"Pausing for {pause_time} seconds...")
        time.sleep(pause_time)

Fetching page 1
Elapsed time: 0:00:23
Estimated remaining time: 1:07:56.214071
Pausing for 62 seconds...
Fetching page 2
Elapsed time: 0:01:49
Estimated remaining time: 1:06:30.956054
Pausing for 62 seconds...
Fetching page 3
Elapsed time: 0:03:11
Estimated remaining time: 1:05:08.383426
Pausing for 62 seconds...
Fetching page 4
Error fetching topics for fengdu78/deeplearning_ai_books: 504 - {"message": "We couldn't respond to your request in time. Sorry about that. Please try resubmitting your request and contact us if the problem persists."}

Elapsed time: 0:04:46
Estimated remaining time: 1:03:33.883563
Pausing for 62 seconds...
Fetching page 5
Elapsed time: 0:06:10
Estimated remaining time: 1:02:09.645374
Pausing for 62 seconds...
Fetching page 6
Elapsed time: 0:07:34
Estimated remaining time: 1:00:45.968749
Pausing for 62 seconds...
Fetching page 7
Elapsed time: 0:08:57
Estimated remaining time: 0:59:22.316064
Pausing for 62 seconds...
Fetching page 8
Elapsed time: 0:10:18
Estimat

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(data)
df
# df['release_tags'] = df['release_dates'].apply(lambda x: [tag for tag, date in x])
# df['release_dates_only'] = df['release_dates'].apply(lambda x: [date for tag, date in x])

# # Explode the release_tags and release_dates_only columns to get each tag and date on its own row
# df_exploded = df.explode(['release_tags', 'release_dates_only'])

# # Rename the columns for clarity
# df_exploded.rename(columns={'release_tags': 'release_tag', 'release_dates_only': 'release_date'}, inplace=True)

# # Drop the original release_dates column as it's no longer needed
# df_exploded.drop(columns=['release_dates'], inplace=True)

# df = df_exploded

# df

,org_name,repo_name,description,repo_url,topics,creation_date,update_date,num_releases,num_stars,num_forks,num_open_issues,num_pull_requests,release_tag,release_date
0,Significant-Gravitas,AutoGPT,AutoGPT is the vision of accessible AI for eve...,https://github.com/Significant-Gravitas/AutoGPT,"ai, gpt-4, openai, python, artificial-intellig...",2023-03-16T09:21:07Z,2024-09-14T17:57:57Z,21,166492,44062,144,4229,autogpt-v0.5.1,2024-04-26T20:15:57Z
0,Significant-Gravitas,AutoGPT,AutoGPT is the vision of accessible AI for eve...,https://github.com/Significant-Gravitas/AutoGPT,"ai, gpt-4, openai, python, artificial-intellig...",2023-03-16T09:21:07Z,2024-09-14T17:57:57Z,21,166492,44062,144,4229,autogpt-v0.5.0,2023-12-14T15:17:16Z
0,Significant-Gravitas,AutoGPT,AutoGPT is the vision of accessible AI for eve...,https://github.com/Significant-Gravitas/AutoGPT,"ai, gpt-4, openai, python, artificial-intellig...",2023-03-16T09:21:07Z,2024-09-14T17:57:57Z,21,166492,44062,144,4229,agbenchmark-v0.0.10,2023-09-17T00:02:08Z
0,Significant-Gravitas,AutoGPT,AutoGPT is the vision of accessible AI for eve...,https://github.com/Significant-Gravitas/AutoGPT,"ai, gpt-4, openai, python, artificial-intellig...",2023-03-16T09:21:07Z,2024-09-14T17:57:57Z,21,166492,44062,144,4229,v0.4.7,2023-08-11T17:55:06Z
0,Significant-Gravitas,AutoGPT,AutoGPT is the vision of accessible AI for eve...,https://github.com/Significant-Gravitas/AutoGPT,"ai, gpt-4, openai, python, artificial-intellig...",2023-03-16T09:21:07Z,2024-09-14T17:57:57Z,21,166492,44062,144,4229,v0.4.6,2023-07-28T12:42:29Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,NEKOparapa,AiNiee,一款专注于Ai翻译的工具，可以用来一键自动翻译RPG SLG游戏，Epub TXT小说，Sr...,https://github.com/NEKOparapa/AiNiee,,2023-04-14T13:58:00Z,2024-09-14T12:14:12Z,30,1191,70,7,61,AiNiee4.62,2024-02-02T12:03:46Z
998,NEKOparapa,AiNiee,一款专注于Ai翻译的工具，可以用来一键自动翻译RPG SLG游戏，Epub TXT小说，Sr...,https://github.com/NEKOparapa/AiNiee,,2023-04-14T13:58:00Z,2024-09-14T12:14:12Z,30,1191,70,7,61,AiNiee-chatgpt4.61,2024-01-15T19:06:28Z
998,NEKOparapa,AiNiee,一款专注于Ai翻译的工具，可以用来一键自动翻译RPG SLG游戏，Epub TXT小说，Sr...,https://github.com/NEKOparapa/AiNiee,,2023-04-14T13:58:00Z,2024-09-14T12:14:12Z,30,1191,70,7,61,AiNiee-chatgpt4.60,2024-01-08T06:25:16Z
998,NEKOparapa,AiNiee,一款专注于Ai翻译的工具，可以用来一键自动翻译RPG SLG游戏，Epub TXT小说，Sr...,https://github.com/NEKOparapa/AiNiee,,2023-04-14T13:58:00Z,2024-09-14T12:14:12Z,30,1191,70,7,61,AiNiee-chatgpt4.59.1,2023-11-24T09:15:57Z


In [ ]:
# Display the DataFrame
# print(df.head(50))
print(df.size)
print(df.shape)
df.to_csv('../data/github_data.csv', index=False)

151942
(10853, 14)
